In [ ]:
#result_path = 'results/pmcsimcyp100/'
result_path = 'results/pmc_sim_ami_128/'

mine_stories = False
min_support = 3

In [ ]:
doc_mapping = {}

first = True
with open(result_path+ 'doc_mapping.tsv', 'r') as f:
    for l in f:
       #skip first line
        if first:
            first = False
            continue
        comp = l.replace('\n','').split('\t')
        snorkel_doc_id = comp[0]
        pubmed_id = comp[1]
        
        doc_mapping[snorkel_doc_id] = pubmed_id
        
print('Amount of document ids: {}'.format(len(doc_mapping)))

In [ ]:
mesh_dict = {}

with open('data/mesh2018.tsv', 'r') as f:
    for l in f: 
        comp = l.replace('\n','').split('\t')
        mesh_id = comp[0]
        mesh_name = comp[1]
        
        mesh_dict[mesh_id] = mesh_name
        
print('Amount of mesh ids: {}'.format(len(mesh_dict)))

In [ ]:
import gzip

gene_dict = {}

first = True
with gzip.open('data/CTD_genes.tsv.gz', 'r') as f:
    for l in f: 
        line = str(l).replace('b\'', '')
        # skip comments
        if line.startswith('#'):
            continue
        #print(line)
        comp = line.replace('\\n','').split('\\t')
        #print(comp)
        gene_id = comp[2]
        gene_name = comp[1]
     
        gene_dict[gene_id] = gene_name
        
print('Amount of gene ids: {}'.format(len(gene_dict)))

In [ ]:
def replace_mesh_id_with_name(mesh_id):
    mesh_id_c = mesh_id
    if mesh_id.startswith('MESH:'):
        mesh_id_c = mesh_id.replace('MESH:','')
    
    if mesh_id_c not in mesh_dict:
        print('Error: Mesh_ID {} not in mesh dict'.format(mesh_id_c))
        return mesh_id
    
    name = mesh_dict[mesh_id_c]
    return name

def replace_gene_id_with_name(gene_id):
    if gene_id not in gene_dict:
        print('Error: Gene_ID {} not in gene dict'.format(gene_id))
        return gene_id
    
    name = gene_dict[gene_id]
    return name

def replace_snorkel_doc_id_with_pubmed_id(doc_id):
    return doc_mapping[doc_id]

In [ ]:
class FactStore:
    def __init__(self):
        self.unique_fact_id_counter = 0
        self.id_to_fact = {}
        self.fact_to_id = {}
        self.doc_to_facts = {}
       
        self.chem_id_to_span = {}
        self.dis_id_to_span = {}
        self.gen_id_to_span = {}
        
    def add_fact(self, doc_id, fact):
        key = frozenset(fact)
        if key in self.fact_to_id:
            unique_fact_id = self.fact_to_id[key]
        else:
            unique_fact_id = self.unique_fact_id_counter
            self.fact_to_id[key] = unique_fact_id
            self.unique_fact_id_counter += 1


        if doc_id not in self.doc_to_facts:
            self.doc_to_facts[doc_id] = set()

        self.doc_to_facts[doc_id].add(unique_fact_id)
        self.id_to_fact[unique_fact_id] = fact
        
    def find_fact_id(fact):
        key = frozenset(fact)
        if key in self.fact_to_id:
            return self.fact_to_id[key]
        return None
        
    def print_info(self):
        print("---------------------------------------")
        print("Amount of ids   : {}".format(len(self.id_to_fact.keys())))
        print("Amount of facts : {}".format(len(self.fact_to_id.keys())))
        print("Amount of docs  : {}".format(len(self.doc_to_facts.keys())))
        print("Known chemicals : {}".format(len(self.chem_id_to_span.keys())))
        print("Known diseases  : {}".format(len(self.dis_id_to_span.keys())))
        print("Known genes     : {}".format(len(self.gen_id_to_span.keys())))
        print("---------------------------------------")
        
    def facts_to_str(self, facts):
        str_res = ""
        str_res += "["
        for f in facts:  
            if 'c_asso_d' is f[1]:
                c_name = replace_mesh_id_with_name(f[0])
                d_name = replace_mesh_id_with_name(f[2])
                str_res += '({}, associated, {})'.format(c_name, d_name) 
            if 'c_inter_g' is f[1]:
                c_name = replace_mesh_id_with_name(f[0])
                g_name = replace_gene_id_with_name(f[2])
                str_res += '({}, interacts, {})'.format(c_name, g_name)
            if 'g_inter_d' is f[1]:
                g_name = replace_gene_id_with_name(f[0])
                d_name = replace_mesh_id_with_name(f[2])
                str_res += '({}, interacts, {})'.format(g_name, d_name)


            if 'c_inhibits_g' is f[1]:
                c_name = replace_mesh_id_with_name(f[0])
                g_name = replace_gene_id_with_name(f[2])
                str_res += '({}, inhibits, {})'.format(c_name, g_name)
            if 'g_metabol_c' is f[1]:
                g_name = replace_gene_id_with_name(f[0])
                c_name = replace_mesh_id_with_name(f[2])
                str_res += '({}, metabol, {})'.format(g_name, c_name)

            str_res += ','

        str_res = str_res[0:-1] +  "]"
        return str_res

        
    def match_query_facts_in_doc_facts(self, query_facts, doc_facts):
        # store all qf substitutions 
        qf_substitutions = {}

        # all query facts must match
        for qf in query_facts:
            # just check whether there is a direct match
            # here no substitution is necessary
            if qf not in doc_facts:
                return (False, {})
            
            # allow variables in query
            if qf[0].startswith('?') or qf[2].startswith('?'):
                # look for possible substitution
                substitutions = []
                for df in doc_facts:
                    # predicates are equal?
                    if qf[1] == df[1]:
                        # is qf[0] not variable?
                        if not qf[0].startswith('?'):
                            # then both must be equal
                            if qf[0] == df[0]:
                                substitutions.append(df)
                            else:
                                # no match
                                break
                        # is qf[2] not variable?
                        if not qf[2].startswith('?'):
                            # then both must be equal
                            if qf[2] == df[2]:
                                substitutions.append(df)
                            else:
                                # no match
                                break
                # no substitution found?
                if len(substitutions) == 0:
                    return (False, {}) # query is not found in documents

                # there is at least one substitution - this fact is matched!
                qf_substitutions[qf] = substitutions
                continue # continue matching process

  
        return (True, qf_substitutions)


    def match_query_facts(self, query_facts):
        number_of_matches = 0
        matched_doc_ids = []
        # go through all documents
        for doc_id, doc_fact_ids in self.doc_to_facts.items():
            # replace all fact_ids by their their original facts
            doc_facts = []
            for dfi in doc_fact_ids:
                doc_facts.append(self.id_to_fact[dfi])

            # now match query against this facts
            (matched, subs) = self.match_query_facts_in_doc_facts(query_facts, doc_facts)
            if matched:
                # match found
                print('Match in {} (PMID: {}) with substitutions:'.format(doc_id, replace_snorkel_doc_id_with_pubmed_id(doc_id)))
                for k, v in subs.items():
                    print('\t{} is substituted by {}\n'.format(k, self.facts_to_str(v)))
                print('\n')
                number_of_matches += 1
                matched_doc_ids.append(doc_id)
        print('{} matches found!'.format(number_of_matches))
        return matched_doc_ids

In [ ]:
fact_store = FactStore()


with open(result_path + 'chemical_disease_association.tsv', 'r') as f:
    first = True
    for line in f:
        if first: # skip header
            first = False
            continue
        
        spl = line.replace('\n', '').split('\t')
        doc_id = spl[0]
        sen_id = spl[1]
        chem_id = spl[3]
        chem_span = spl[4]
        dis_id = spl[5]
        dis_span = spl[6]
        
        fact_store.chem_id_to_span[chem_id] = chem_span
        fact_store.dis_id_to_span[dis_id] = dis_span
        
        #fact = (chem_span, 'cd', dis_span)      
        fact = (chem_id, 'c_asso_d', dis_id)
        fact_store.add_fact(doc_id, fact)

fact_store.print_info()

with open(result_path + 'chemical_gene_interaction.tsv', 'r') as f:
    first = True
    for line in f:
        if first: # skip header
            first = False
            continue
        
        spl = line.replace('\n', '').split('\t')
        doc_id = spl[0]
        sen_id = spl[1]
        chem_id = spl[3]
        chem_span = spl[4]
        gen_id = spl[5]
        gen_span = spl[6]
        
        fact_store.chem_id_to_span[chem_id] = chem_span
        fact_store.gen_id_to_span[gen_id] = gen_span
                
        #fact = (chem_span, 'inh', gen_span)
        fact = (chem_id, 'c_inter_g', gen_id)
        fact_store.add_fact(doc_id, fact)

        
fact_store.print_info()

with open(result_path + 'gene_disease_interaction.tsv', 'r') as f:
    first = True
    for line in f:
        if first: # skip header
            first = False
            continue
        
        spl = line.replace('\n', '').split('\t')
        doc_id = spl[0]
        sen_id = spl[1]
        gen_id = spl[3]
        gen_span = spl[4]
        dis_id = spl[5]
        dis_span = spl[6]
        
        fact_store.gen_id_to_span[gen_id] = gen_span
        fact_store.dis_id_to_span[dis_id] = dis_span
               
        #fact = (chem_span, 'inh', gen_span)
        fact = (gen_id, 'g_inter_d', dis_id)
        fact_store.add_fact(doc_id, fact)

fact_store.print_info()

with open(result_path + 'chemical_gene_inhibition.tsv', 'r') as f:
    first = True
    for line in f:
        if first: # skip header
            first = False
            continue
        
        spl = line.replace('\n', '').split('\t')
        doc_id = spl[0]
        sen_id = spl[1]
        chem_id = spl[3]
        chem_span = spl[4]
        gen_id = spl[5]
        gen_span = spl[6]
        
        fact_store.gen_id_to_span[gen_id] = gen_span
        fact_store.chem_id_to_span[chem_id] = chem_span
               
        fact = (chem_id, 'c_inhibits_g', gen_id)
        fact_store.add_fact(doc_id, fact)

        
fact_store.print_info()

with open(result_path + 'gene_chemical_metabolism.tsv', 'r') as f:
    first = True
    for line in f:
        if first: # skip header
            first = False
            continue
        
        spl = line.replace('\n', '').split('\t')
        doc_id = spl[0]
        sen_id = spl[1]
        gen_id = spl[3]
        gen_span = spl[4]
        chem_id = spl[5]
        chem_span = spl[6]
        
        fact_store.gen_id_to_span[gen_id] = gen_span
        fact_store.chem_id_to_span[chem_id] = chem_span
               
        fact = (gen_id, 'g_metabol_c', chem_id)
        fact_store.add_fact(doc_id, fact)
        
        
        
        
fact_store.print_info()


Computing frequent occurring facts

In [ ]:
if mine_stories:
    to_check = []
    ids_with_min_support = set()
    for f_id in fact_store.id_to_fact.keys():
        support = 0
        # go through all documents
        for doc_facts in fact_store.doc_to_facts.values():
            if f_id in doc_facts:
                 support += 1

        if support >= min_support:
            t_set = set()
            t_set.add(f_id)
            to_check.append(t_set)
            ids_with_min_support.add(f_id)
    print(ids_with_min_support)

Computing frequent item sets 

In [ ]:
if mine_stories:
    results = []

    explored_sets = set()
    while to_check:
        # get fact candidate ids
        cand_ids_org = to_check.pop()
        for f_id in ids_with_min_support:
            if f_id in cand_ids_org:
                continue
            # check with this id included
            cand_ids = cand_ids_org.copy()
            cand_ids.add(f_id)

            # already checked this combi
            if frozenset(cand_ids) in explored_sets:
                continue

            #print("Starting with candidate ids: {}".format(cand_ids_org))
            # how much support does these ids have?
            support = 0
            # go through all documents
            doc_ids_supporting = []
            for doc_id, doc_facts in fact_store.doc_to_facts.items():
                included = True
                for f_id in cand_ids:
                    if f_id not in doc_facts:
                        # if a fact id is not included - stop here (no support)
                        included = False
                        break
                if included:
                    doc_ids_supporting.append(doc_id)
                    support += 1

            explored_sets.add(frozenset(cand_ids))

            if support >= min_support:
                results.append((cand_ids.copy(), support, doc_ids_supporting))
                to_check.append(cand_ids)
                print("Support {} for {} in doc_ids: {}".format(support, cand_ids, doc_ids_supporting))



In [ ]:
if mine_stories:
    stories = sorted(results, key=lambda x: x[1], reverse=True)

    for res, support, doc_ids in stories:
        facts = []
        for f_id in res:
            facts.append(fact_store.id_to_fact[f_id])
        print("Support {} for {}\n".format(support,fact_store.facts_to_str(facts)))

In [ ]:
if mine_stories:
    len(stories)

In [ ]:
if mine_stories:
    filename = result_path + 'stories_supp{}.tsv'.format(min_support)
    with open(filename, 'w') as f:
        #f.write('{}\t{}\n'.format('support', 'frequent item set'))
        for res, support, doc_ids in stories:
            facts = set()
            for f_id in res:
                facts.add(fact_store.id_to_fact[f_id])

            # translate documument id
            translated_doc_ids = []
            for doc_id in doc_ids:
                translated_doc_ids.append(replace_snorkel_doc_id_with_pubmed_id(doc_id))


            f.write('{}\t{}\t{}\n'.format(support,facts, translated_doc_ids))
    print('Stories saved at {}'.format(filename))

In [ ]:
if mine_stories:
    filename = result_path + 'stories_supp{}_translated.tsv'.format(min_support)
    with open(filename, 'w') as f:
        for story, supp, doc_ids in stories:
            line = '{}'.format(supp)

            # translate id to facts
            facts = set()
            for f_id in story:
                facts.add(fact_store.id_to_fact[f_id])

            for event in facts:
                pred = event[1]

                if pred == 'c_asso_d':
                    ev1 = replace_mesh_id_with_name(event[0])
                    ev2 = replace_mesh_id_with_name(event[2])
                elif pred == 'c_inter_g':
                    ev1 = replace_mesh_id_with_name(event[0])
                    ev2 = replace_gene_id_with_name(event[2])
                elif pred == 'g_inter_d':
                    ev1 = replace_gene_id_with_name(event[0])
                    ev2 = replace_mesh_id_with_name(event[2])
                elif pred == 'c_inhibits_g':
                    ev1 = replace_mesh_id_with_name(event[0])
                    ev2 = replace_gene_id_with_name(event[2])
                elif pred == 'g_metabol_c':
                    ev1 = replace_gene_id_with_name(event[0])
                    ev2 = replace_mesh_id_with_name(event[2])

                line += '\t({},{},{})'.format(ev1, pred, ev2)

            # translate documument id
            translated_doc_ids = []
            for doc_id in doc_ids:
                translated_doc_ids.append(replace_snorkel_doc_id_with_pubmed_id(doc_id))

            line += '\t{}'.format(translated_doc_ids)

            line += '\n'
            f.write(line)
    print('Translated stories saved at {}'.format(filename))

In [ ]:
# Sim -- asso -- Rhabdo, CYP3A4 -- meta -- Simvastatn, ?X -- inhib -- 1576
#query = [('MESH:D019821', 'c_asso_d', 'MESH:D012206'), ('1576', 'g_metabol_c', 'MESH:D019821'), ('?X', 'c_inhibits_g', '1576')]

#query = [('1576', 'g_metabol_c', 'MESH:D019821'), ('?X', 'c_inhibits_g', '1576')]


# CYP3A4 - meta - Simvastatin, Erythromycin - inhibits - CYP3A4
#query = [('1576', 'g_metabol_c', 'MESH:D019821'), ('MESH:D004917', 'c_inhibits_g', '1576')]

# CYP3A4 - meta - Simvastatin, Amiodarone - inhibits - CYP3A4
query = [('1576', 'g_metabol_c', 'MESH:D019821'), ('MESH:D000638', 'c_inhibits_g', '1576')]


%time query_doc_ids = fact_store.match_query_facts(query)

print(query_doc_ids)

In [ ]:
for d_id in query_doc_ids[0:1]:
    facts = []
    for f_id in fact_store.doc_to_facts[d_id]:
        facts.append(fact_store.id_to_fact[f_id])
    print("Story for PMID {}: {}\n\n".format(replace_snorkel_doc_id_with_pubmed_id(d_id), fact_store.facts_to_str(facts)))

In [ ]:
import random

# pick a sample of random documents
sample_size = 25

# sample of document ids
sample_doc_ids = random.choices(list(fact_store.doc_to_facts.keys()), k=sample_size)

print(sample_doc_ids)


filename = result_path + 'graph_queried_documents.tsv'
with open(filename, 'w') as f:
    f.write('Document ID\tMatch?')
    for doc_id in sample_doc_ids:
        pmid = replace_snorkel_doc_id_with_pubmed_id(doc_id)
        if doc_id in query_doc_ids:
            # contains match
            f.write('\nPMC{}\tMatch'.format(pmid))
        else:
            # contains no match
            f.write('\nPMC{}\tNo Match'.format(pmid))
